## Noise Magnitude vs Coefficient Estimate Error

Tests the effect of Gaussian white noise on the estimated coefficients 

In [1]:
%load_ext autoreload
%autoreload 2

# Import Python packages
import pickle

# Package Imports
from sindy_bvp import SINDyBVP
from sindy_bvp.differentiators import PolyInterp
from sindy_bvp.library_builders import NoiseMaker

In [2]:
# Set file to load and stem for saving
load_stem = "./data/S4-EB-"
save_stem = "./data/Fig5a-S4-"

In [3]:
%%time

# Set a range of noise magnitudes to test
noise_magnitudes = [0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.075, 0.10]

# Since the data is noisy, we'll use a Polynomial Interpolation derivative method
poly = PolyInterp(diff_order=4, width=30, degree=6)

# Create an empty
results_list = []

print("Completed:", end=" ")
for noise_mag in noise_magnitudes:
    # Initialize NoiseMaker, which adds noise then filters noisy signal
    nm = NoiseMaker(noise_magnitude=noise_mag)

    # Initialize SINDyBVP object
    sbvp = SINDyBVP(file_stem = load_stem,
                    num_trials = 200,
                    differentiator = poly,
                    outcome_var = 'd^{4}u/dx^{4}',
                    noisemaker = nm,
                    known_vars = ['u', 'u^{2}', 'du/dx', 'f'],
                    dep_var_name = 'u',
                    ind_var_name = 'x')

    # Execute the optimization
    coeffs, plotter = sbvp.sindy_bvp()

    # Compute the S-L coeffs with Plotter analysis tool
    plotter.compute_sl_coeffs()
    
    # gather the learned coefficients and relevant metrics
    # And place into the results_list
    results_list.append({'noise_mag': noise_mag,
                         'loss': min(sbvp.groupreg.Losses),
                         'p': plotter.inferred_phi, 
                         'q': plotter.inferred_q,
                         'coeffs': coeffs})
    
    print(noise_mag, end=" | ")

Completed: 0.001 | 0.005 | 0.01 | 0.02 | 0.03 | 0.04 | 0.05 | 0.075 | 0.1 | Wall time: 28min


In [4]:
## Pickle the results
pickle.dump(results_list, open(save_stem+"results.pickle", "wb"))